# Evolutionary Computation for Evolving Neural Network EMG Classifier Structure and Weights
In this project, the goal is to improve the structure and weights of a neural network that classifies EMG signals, and compare the genetic solutions to standard classifiers.

Controls:
- Random Forest Classifier
- Neural Network (Backpropagation)

Evolutionary Models:
- NEAT (NeuroEvolution of Augmenting Topologies)
- HyperNEAT

In [1]:
%load_ext autoreload
%autoreload 2

In [16]:
# Imports
from __future__ import annotations
from pathlib import Path
from typing import List, Tuple, Dict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

# Custom library
from emg_evo_lib_kb import *

In [32]:
# Dataset root
root_path = Path("/Users/kaanborekci/Desktop/CS496/Code/Self-Project-Evolutionary-Computation/EMG-Project/UC-Irvine-Set/EMG_data_for_gestures-master")

# Column names in each raw file
EMG_COLUMNS = ["Time"] + [f"ch{i}" for i in range(1, 9)] + ["Class"]

# Gesture classes we care about (ignore 0 = unmarked)
VALID_GESTURE_CLASSES = [1, 2, 3, 4, 5, 6, 7]

In [33]:
files = iter_emg_files(root_path, pattern="*.txt")
files[0]

PosixPath('/Users/kaanborekci/Desktop/CS496/Code/Self-Project-Evolutionary-Computation/EMG-Project/UC-Irvine-Set/EMG_data_for_gestures-master/01/1_raw_data_13-12_22.03.16.txt')

In [30]:
df = load_emg_file(files[2], EMG_COLUMNS)

/Users/kaanborekci/Desktop/CS496/Code/Self-Project-Evolutionary-Computation/EMG-Project/Evolutionary-EMG-Classification/src/emg_evo_lib_kb/preprocessing.py:22: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, delim_whitespace=True, header=0, names=emg_column_names)


In [31]:
df

,Time,ch1,ch2,ch3,ch4,ch5,ch6,ch7,ch8,class
0,time,channel1,channel2,channel3,channel4,channel5,channel6,channel7,channel8,class
1,1,-1e-05,-3e-05,-2e-05,0,-2e-05,-1e-05,0,-2e-05,0
2,4,-1e-05,-3e-05,-2e-05,0,-2e-05,-1e-05,0,-2e-05,0
3,5,-1e-05,-3e-05,-2e-05,0,-2e-05,-1e-05,0,-2e-05,0
4,6,-1e-05,-3e-05,-2e-05,0,-2e-05,-1e-05,0,-2e-05,0
...,...,...,...,...,...,...,...,...,...,...
72318,74521,-0.00003,-0.00003,0.00002,-0.00001,-0.00002,0.00012,0.00004,-0.00001,0
72319,74522,-0.00003,-0.00003,0.00002,-0.00001,-0.00002,0.00012,0.00004,-0.00001,0
72320,74523,-0.00003,-0.00003,0.00002,-0.00001,-0.00002,0.00012,0.00004,-0.00001,0
72321,74524,-0.00003,-0.00003,0.00002,-0.00001,-0.00002,0.00012,0.00004,-0.00001,0
